In [152]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import urllib.request
import sys
import pyodbc
from psycopg2 import connect

In [141]:
df_new1= pd.read_parquet('UK_local_and_regional_CO2_emissions.parquet', engine='pyarrow') 
df_new2= pd.read_parquet('UK_local_and_regional_CO2_emissions_pollution_inventory.parquet', engine='pyarrow') 
df_new4= pd.read_parquet('details.parquet', engine='pyarrow') 

In [142]:
df_new2 = df_new2.iloc[:, [0,1,2,3,4, 5,20]]
df_new2 = df_new2[df_new2['2019 year'] != '0.0']
df_new2

,Local Authority Distract Name,Operator,Site,Postcode,Reference,Substance Name,2019 year
0,Aberdeen City,Arjo Wiggins Ltd,Stoneywood Mill,AB21 9AB,0,Carbon dioxide,57.722
5,Aberdeenshire,ExonMobil,SAGE,0,0,Carbon dioxide,94.324997
8,Aberdeenshire,National Grid Gas Plc,Aberdeen,AB32 6US,0,Carbon dioxide,26.857664
9,Aberdeenshire,National Grid Gas Plc,St Fergus,AB42 3EP,0,Carbon dioxide,26.367117
10,Aberdeenshire,Scottish & Southern Energy Plc,Peterhead,AB42 3BZ,IPC/N/20019,Carbon dioxide,1578.535572
...,...,...,...,...,...,...,...
2561,Wychavon,Severn Waste Services Ltd,Mercia Envirecover Site,DY10 4JB,XP3935TX,Carbon dioxide,129.766
2570,Wyre,Sita (Lancashire) Ltd,Jameson Road Landfill Site,FY7 8TW,GP3937PZ,Carbon dioxide,19.5
2571,Wyre,Victrex Manufacturing Ltd,Thornton-Cleveleys,FY5 4QD,BU5640IA,Carbon dioxide,19.1
2573,Wyre,Vinnolit Hillhouse Ltd,Thornton-Cleveleys,FY5 4QD,TP3833GG,Carbon dioxide,11.64


In [143]:
df_merge1 = pd.merge(df_new1[df_new1['Year'] == 2019].iloc[:, :11], df_new2, how='inner',left_on='Local Authority', right_on='Local Authority Distract Name')
#df_merge1.to_parquet("merged.parquet")

In [144]:
df_merge1['2019 year'] = df_merge1['2019 year'].astype('float')


In [145]:
df_merge1['label'] = np.where(
    df_merge1.iloc[:,6] == df_merge1['2019 year']/2, 0, np.where(
    df_merge1.iloc[:,6] >  df_merge1['2019 year']/2, 1, -1)) 

In [149]:
df_merge1.to_parquet('merged.parquet')

In [147]:
df_merged= pd.read_parquet('merged.parquet', engine='pyarrow') 

In [150]:
df_merge1

,Region/Country,Second Tier Authority,Local Authority,Code,Year,Industry Electricity,Industry Gas,Industry 'Other Fuels',Large Industrial Installations,Agriculture,Industry Total,Local Authority Distract Name,Operator,Site,Postcode,Reference,Substance Name,2019 year,label
0,North East,Darlington,Darlington,E06000005,2019,17.786253,64.760412,23.930058,0.050426,5.952103,112.479252,Darlington,Ineos ChlorVinyls Ltd,Newton Aycliffe,DL5 6EA,FP3833NP,Carbon dioxide,49.343000,1
1,North East,Durham,County Durham,E06000047,2019,110.906985,86.680235,130.746196,10.899522,45.994055,385.226992,County Durham,Dalkia Bio Energy Ltd,Chilton Biomass Plant,DL17 0SD,BP3736HA,Carbon dioxide,0.043075,1
2,North East,Durham,County Durham,E06000047,2019,110.906985,86.680235,130.746196,10.899522,45.994055,385.226992,County Durham,Lucite International UK Ltd,Newton Aycliffe,DL5 6YE,CP3335LZ,Carbon dioxide,4.700560,1
3,North East,Gateshead,Gateshead,E08000037,2019,45.752494,76.066494,35.248639,4.795098,1.542726,163.405451,Gateshead,Octagon Green Solutions Ltd,Blaydon Quarry,NE21 4SX,WP3537ZG,Carbon dioxide,22.400000,1
4,North East,Gateshead,Gateshead,E08000037,2019,45.752494,76.066494,35.248639,4.795098,1.542726,163.405451,Gateshead,Sita Environment Ltd,Path Head Quarry Landfill,NE21 4SP,BJ7824IK,Carbon dioxide,22.200000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
619,Northern Ireland,Northern Ireland,Mid and East Antrim,N09000008,2019,38.559634,56.107100,217.406073,11.847051,45.652023,369.571881,Mid and East Antrim,Kilroot Power Ltd,Kilroot,BT38 7LX,0120/06A,Carbon dioxide,1010.033350,-1
620,Northern Ireland,Northern Ireland,Mid and East Antrim,N09000008,2019,38.559634,56.107100,217.406073,11.847051,45.652023,369.571881,Mid and East Antrim,Premier Power Ltd,Ballylumford,BT40 3RS,0125/06A,Carbon dioxide,1141.294000,-1
621,Northern Ireland,Northern Ireland,Mid Ulster,N09000009,2019,75.808282,24.178805,349.697965,357.242559,92.253520,899.181131,Mid Ulster,Dale Farm Ltd,Dunmanbridge,BT80 9UU,0094/05A,Carbon dioxide,18.977000,1
622,Northern Ireland,Northern Ireland,Mid Ulster,N09000009,2019,75.808282,24.178805,349.697965,357.242559,92.253520,899.181131,Mid Ulster,Lafarge Cement (Ireland) Ltd,Cookstown,BT80 9AR,0052/04A,Carbon dioxide,313.326000,-1


In [155]:
conn = connect(
    dbname = "Initialdatabase",
    user = "shihao",
    host = "de166.c0imhubqi7yg.us-east-1.rds.amazonaws.com",
    password = "qwerty123")
cur = conn.cursor()
count =0
for index, row in df_merge1.iterrows():
    cur.execute("INSERT INTO public.merged (RegionCountry,SecondTierAuthority, LocalAuthority, Code, Year, IndustryElectricity, IndustryGas, IndustryOtherFuels, LargeIndustrialInstallations, Agriculture, IndustryTotal, LocalAuthorityDistractName, Operator,Site,Postcode,Reference,SubstanceName,year2019,label) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)",(row[0], row[1],row[2],row[3],row[4],row[5],row[6],row[7],row[8],row[9],row[10],row[11],row[12],row[13],row[14],row[15],row[16],row[17],row[18]))
    conn.commit()
    count +=1
    print("success"+str(count))    
cur.close()
conn.close()

success1
success2
success3
success4
success5
success6
success7
success8
success9
success10
success11
success12
success13
success14
success15
success16
success17
success18
success19
success20
success21
success22
success23
success24
success25
success26
success27
success28
success29
success30
success31
success32
success33
success34
success35
success36
success37
success38
success39
success40
success41
success42
success43
success44
success45
success46
success47
success48
success49
success50
success51
success52
success53
success54
success55
success56
success57
success58
success59
success60
success61
success62
success63
success64
success65
success66
success67
success68
success69
success70
success71
success72
success73
success74
success75
success76
success77
success78
success79
success80
success81
success82
success83
success84
success85
success86
success87
success88
success89
success90
success91
success92
success93
success94
success95
success96
success97
success98
success99
success100
success1